<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#责任链模式:-一个简单的例子" data-toc-modified-id="责任链模式:-一个简单的例子-0.0.1"><span class="toc-item-num">0.0.1&nbsp;&nbsp;</span>责任链模式: 一个简单的例子</a></span></li></ul></li></ul></li><li><span><a href="#Chain-of-Responsibility-Pattern" data-toc-modified-id="Chain-of-Responsibility-Pattern-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Chain of Responsibility Pattern</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#a-conventional-chain" data-toc-modified-id="a-conventional-chain-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>a conventional chain</a></span></li><li><span><a href="#A-Coroutine-Based-Chain" data-toc-modified-id="A-Coroutine-Based-Chain-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>A Coroutine-Based Chain</a></span></li></ul></li></ul></li></ul></div>

###### 责任链模式: 一个简单的例子
创建链式对象用来接受广播消息:

* 我们把消息发送给一系列对象的首个节点，对象可以选择处理消息或者向下一个对象传递,只有对消息感兴趣的节点处理。
* 用来解耦发送者和接收者。
* 在python里通过dynamic dispatching来实现。

In [1]:
# 一个事件驱动系统
class Event(object):

    def __init__(self, name):

        self.name = name

    def __str__(self):

        return self.name


class Widget(object):
    """组件的基类
    """

    def __init__(self, parent=None):

        self.parent = parent

    def handle(self, event):
        """ dynamic dispatching
        """
        handler = f'handle_{event}'
        if hasattr(self, handler):
            method = getattr(self, handler)
            method(event)
        elif self.parent is not None:
            self.parent.handle(event)
        elif hasattr(self, 'handle_default'):
            self.handle_default(event)
        else:
            raise NotImplementedError

# 子类负责实现具体的操作


class MainWindow(Widget):

    def handle_close(self, event):

        print(f'MainWindow: {event}')

    def handle_default(self, event):

        print(f'MainWindow: Default {event}')


class SendDialog(Widget):

    def handle_paint(self, event):

        print(f'SendDialog: {event}')


class MsgText(Widget):

    def handle_down(self, event):

        print(f'MsgText: {event}')

In [2]:
# 创建链式对象
main_window = MainWindow()
send_dialog = SendDialog(main_window)
msg_text = MsgText(send_dialog)

for e in ('down', 'paint', 'unhandled', 'close'):
    event = Event(e)
    print('\nSending event -{}- to MainWindow'.format(event))
    main_window.handle(event)
    print('Sending event -{}- to SendDialog'.format(event))
    send_dialog.handle(event)
    print('Sending event -{}- to MsgText'.format(event))
    msg_text.handle(event)


Sending event -down- to MainWindow
MainWindow: Default down
Sending event -down- to SendDialog
MainWindow: Default down
Sending event -down- to MsgText
MsgText: down

Sending event -paint- to MainWindow
MainWindow: Default paint
Sending event -paint- to SendDialog
SendDialog: paint
Sending event -paint- to MsgText
SendDialog: paint

Sending event -unhandled- to MainWindow
MainWindow: Default unhandled
Sending event -unhandled- to SendDialog
MainWindow: Default unhandled
Sending event -unhandled- to MsgText
MainWindow: Default unhandled

Sending event -close- to MainWindow
MainWindow: close
Sending event -close- to SendDialog
MainWindow: close
Sending event -close- to MsgText
MainWindow: close


#### Chain of Responsibility Pattern

Instead of one function directly calling another, the first function sends a request to a chain of receivers. 

The first receiver in the chain either can handle the request and stop the chain (by not passing the request on) or can pass on the request to the next receiver in the chain.

###### a conventional chain

event --> TimerHandler --> KeyHandler --> MouseHandler --> NullHandler

In [45]:
import sys
from enum import Enum

Event = Enum('Event', 'MOUSE KEY TIMER DISCARD TERMINATE')


class NullHandler(object):
    """Handler 的基类, Handler 的最末端
    """
    __event = Event.DISCARD

    def __init__(self, successor=None):
        """successor 是下一个 Handler 的实例
        """
        self.__successor = successor

    def handle(self, event):
        """
        首先判断 event 类型是不是自己能够处理的, 如果不是并且
        有后继 Handler就调用后继 Handler 的 handle 方法.
        """
        if event.kind == __event:
            # Do something here
            print('Click: {}'.format(event))
            self._handle()
        elif self.__successor is not None:
            self.__successor.handle(event)
        else:
            raise NotImplementedError

    def _handle(self):

        raise NotImplementedError


class MouseHandler(NullHandler):

    __event = Event.MOUSE

    def _handle(self):
        pass


class KeyHandler(NullHandler):

    __event = Event.KEY

    def _handle(self):
        pass


class TimerHandler(NullHandler):

    __event = Event.TIMER

    def _handle(self):
        pass


class DebugHandler(NullHandler):
    """DebugHandler 不处理任何事件
    """

    def __init__(self, successor=None, file=sys.stdout):

        super().__init__(successor)
        self.__file = file

    def handle(self, event):

        self.__file.write(f'*DEBUG* : {event}\n')
        super().handle(event)

###### A Coroutine-Based Chain

In [46]:
import functools


def coroutine(func):
    """协程函数装饰器, 自动启动生成器函数
    """
    @functools.wraps(func)
    def wrapper(*args, **kwargs):  
        generator = func(*args, **kwargs)
        next(generator)
        return generator
    
    return wrapper

In [51]:
# 这样我们的 生成器函数 这么写


@coroutine
def key_handler(successor=None):

    while True:
        event = (yield)
        if event == Event.KEY:
            print(f'Input: {event}')
            # Do something here
        elif successor is not None:
            successor.send(event)
        else:
            raise NotImplementedError


@coroutine
def null_handler(successor=None):
    while True:
        event = (yield)
        if event == Event.DISCARD:
            print(f'Discard: {event}')
            # Do something here
        elif successor is not None:
            successor.send(event)
        else:
            raise NotImplementedError
    


@coroutine
def time_handler(successor=None):
    while True:
        event = (yield)
        if event == Event.TIMER:
            print(f'Timing: {event}')
            # Do something here
        elif successor is not None:
            successor.send(event)
        else:
            raise NotImplementedError


@coroutine
def mouse_handler(successor=None):
    while True:
        event = (yield)
        if event == Event.MOUSE:
            print(f'Press: {event}')
            # Do something here
        elif successor is not None:
            successor.send(event)
        else:
            raise NotImplementedError

In [52]:
pipeline = key_handler(mouse_handler(time_handler(null_handler(None))))

In [56]:
Event_ = iter(Event)

# 事件循环
while True:
    event = next(Event_)
    if event == Event.TERMINATE:
        break
    pipeline.send(event)

Press: Event.MOUSE
Input: Event.KEY
Timing: Event.TIMER
Discard: Event.DISCARD
